<a href="https://colab.research.google.com/github/pkonowrocki/Weather-prediction/blob/master/ClassificationModelsComp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
a = torch.Tensor([1,2,3])
b = a>2
print(a>2)
print(b==1)

tensor([False, False,  True])
tensor([False, False,  True])


In [0]:
#boilerplate code
import subprocess
file_id = '1XfiON89EFCsw5zhtD4hq1hn5Z21vGzpG'
subprocess.run(['pip', 'install', 'PyDrive'])
subprocess.run(['apt-get', 'install', 'unzip'])
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import os
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
downloaded = drive.CreateFile({'id':file_id})
downloaded.GetContentFile('Dataset.zip')
if not os.path.exists('./Dataset'):
  subprocess.run(['unzip', './Dataset.zip'])
print(f'{"".join(["-" for _ in range(10)])}DATA READY{"".join(["-" for _ in range(10)])}')

#code
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
from sklearn import preprocessing
from sklearn import metrics
import math


class GELU(nn.Module):
  def __init__(self):
    super(GELU, self).__init__()

  def forward(self, x):
    x = 0.5*x*(1+torch.tanh(math.sqrt(2/math.pi)*(x+0.044715*torch.pow(x, 3))))
    return x

if torch.cuda.is_available():
    device = torch.device('cuda:0')
    print('cuda')
else:
    device = torch.device('cpu')
    print('cpu')

def testNetwork(Xtest, Ytest, net, criterion):
  x = torch.Tensor(Xtest).to(device)
  output = net(x)
  yTrue = torch.Tensor(np.array([Ytest]).T).to(device)
  loss = criterion(output, yTrue).to(device)
  tn, fp, fn, tp = metrics.confusion_matrix((yTrue==1).cpu().detach().numpy(), (output>=0.5).cpu().detach().numpy()).ravel()
  
  return torch.mean(loss).cpu().detach().numpy(), torch.std(loss).cpu().detach().numpy(), tn, fp, fn, tp

def trainNetwork(path, code, net = None, epochs = None, criterion = None, checkEvery = None, optimizer = None, val = 0.0, scaler = None, batch = 1, gradClipping = 0, seed = 0):
  torch.manual_seed(seed)
  np.random.seed(seed)

  Xtrain = np.genfromtxt(f'{path}/{code}-train-input.csv', delimiter=',')
  Ytrain = np.genfromtxt(f'{path}/{code}-train-output.csv', delimiter=',')
  Xtest = np.genfromtxt(f'{path}/{code}-test-input.csv', delimiter=',')
  Ytest = np.genfromtxt(f'{path}/{code}-test-output.csv', delimiter=',')
  if scaler is None:
    scaler = preprocessing.StandardScaler().fit(Xtrain)
  Xtrain = scaler.transform(Xtrain)
  Xtest = scaler.transform(Xtest)

  if not val == 0.0:
    n = int(len(Ytrain)*val)+1 if int(len(Ytrain)*val)+1 < len(Ytrain) else int(len(Ytrain)*val)
    idxs = np.random.choice(np.arange(0, len(Ytrain), 1), n)
    Xval = Xtrain[idxs]
    Yval = Ytrain[idxs]
    Xtrain = np.delete(Xtrain, idxs, axis=0)
    Ytrain = np.delete(Ytrain, idxs)
  else:
    Xval = None

  if epochs is None:
    epochs = 500
  if checkEvery is None:
    checkEvery = 5
  if net is None:
    numInputs = Xtrain.shape[1]
    numOutputs = 1
    hiddenSize = 50
    net = nn.Sequential(
      nn.Linear(numInputs, 2*numInputs),
      GELU(),
      nn.Linear(2*numInputs, numInputs),
      GELU(),
      nn.Linear(numInputs, numInputs),
      GELU(),
      nn.Linear(numInputs, numOutputs)).to(device)
  if criterion is None:
    criterion = nn.MSELoss(reduction='none')
  if optimizer is None:
    optimizer = optim.Adadelta(net.parameters())

  net.to(device)

  if batch == 1:
    splitXtrain = Xtrain
    splitYtrain = Ytrain
  else:
    batch = int(len(Ytrain)/batch)
    splitXtrain = np.array_split(Xtrain, batch)
    splitYtrain = np.array_split(Ytrain, batch)

  for epoch in range(epochs):
    runningLoss = 0.0
    for i, x in enumerate(splitXtrain, 0):
      if np.any(np.isnan(x)):
        raise('NaN value in input')
      if np.any(np.isnan(splitYtrain[i])):
        raise('NaN value in target')
      optimizer.zero_grad()
      y = torch.Tensor(np.array([splitYtrain[i]]).T).to(device)
      x = torch.Tensor(x).to(device)
      output = net(x)
      if np.any(np.isnan(output.cpu().detach().numpy())):
        raise('NaN value in output')
      loss = torch.mean(criterion(output, y))
      loss.backward()
      if not gradClipping == 0:
        torch.nn.utils.clip_grad_norm_(net.parameters(), gradClipping)
      optimizer.step()
      runningLoss += loss.item()
    
    if epoch % checkEvery == checkEvery-1 or epoch == 0:
      net.eval()
      if not Xval is None:
        lossVal, stdVal, tnVal, fpVal, fnVal, tpVal  = testNetwork(Xval, Yval, net, criterion)
      
      lossTest, stdTest, tnTest, fpTest, fnTest, tpTest = testNetwork(Xtest, Ytest, net, criterion)
      print(f'{epoch}, {runningLoss/len(splitYtrain)}, {lossTest}, {stdTest}, {tnTest}, {fpTest}, {fnTest}, {tpTest}{", " + str(lossVal) +", " + str(stdVal)+", "+str(tnVal)+", "+str(fpVal)+", "+str(fnVal)+", "+str(tpVal) if not Xval is None else ""}')
      net.train()
  
  net.eval()


for i in range(100):
  numInputs = 60
  seed = np.random.randint(np.iinfo(np.int32).max)
  print(f'{i} GELU (seed: {seed})')
  trainNetwork('./Dataset/Wind/NoImputation', 'CloudsOktaWindNatural', val = 0.2, batch = 32, gradClipping = 2, seed = seed, net = nn.Sequential(
      nn.Linear(numInputs, 4*numInputs),
      GELU(),
      nn.Linear(4*numInputs, 3*numInputs),
      GELU(),
      nn.Linear(3*numInputs, 2*numInputs),
      GELU(),
      nn.Linear(2*numInputs, numInputs),
      GELU(),
      nn.Linear(numInputs, 1),
      nn.Sigmoid()).to(device))
  print(f'{i} TANH (seed: {seed})')
  trainNetwork('./Dataset/Wind/NoImputation', 'CloudsOktaWindNatural', val = 0.2, batch = 32, gradClipping = 2, seed = seed, net = nn.Sequential(
      nn.Linear(numInputs, 4*numInputs),
      nn.Tanh(),
      nn.Linear(4*numInputs, 3*numInputs),
      nn.Tanh(),
      nn.Linear(3*numInputs, 2*numInputs),
      nn.Tanh(),
      nn.Linear(2*numInputs, numInputs),
      nn.Tanh(),
      nn.Linear(numInputs, 1),
      nn.Sigmoid()).to(device))

----------DATA READY----------
cuda
0 GELU (seed: 451909655)
0, 0.05077741386258944, 0.03157113119959831, 0.1378328651189804, 1780, 0, 56, 0, 0.039401554, 0.15353757, 2013, 0, 89, 0
4, 0.03182020551574505, 0.028543882071971893, 0.13752004504203796, 1780, 0, 56, 0, 0.036872964, 0.15667692, 2013, 0, 89, 0
9, 0.030897885182888347, 0.028236443176865578, 0.13423699140548706, 1780, 0, 56, 0, 0.037051775, 0.15591414, 2013, 0, 89, 0
14, 0.029749128978445376, 0.029066238552331924, 0.13287334144115448, 1779, 1, 56, 0, 0.038146246, 0.15722752, 2010, 3, 88, 1
